In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from bokeh.io import output_notebook
from bokeh.plotting import figure, show,output_file
from bokeh.layouts import row
from bokeh.models import Panel, Tabs
from bokeh.models import Range1d
import math

In [2]:
class System:
    def __init__(self, n, m, a, b, inp_type,t=10):
        """
        initializing the parameters
        
        """
        
        self.n = n + 1
        self.m = m + 1
        self.end_t = t
        self.k = 1000
        self.h = (self.end_t - 0) / self.k
        self.a = np.array(a)/a[0] 
        self.b = np.array(b)/a[0]
        self.inp_type = inp_type
        self.time = np.linspace(0, self.end_t, self.k + 1)
        self.u = np.ones(self.k+1)
        
    def computeStateSpace(self):
        """
        Turning the inp/op eqaution to state space representation
        to be easy to be solved using numerical analysis
        
        """
        
        self.a = self.a[::-1]
        self.b = self.b[::-1]
        self.A = np.zeros((self.n - 1, self.n - 1))
        self.Beta = np.zeros(self.n)
        self.C = np.zeros(self.n - 1)
        self.C[0] = 1
        self.D = self.a[self.n - 1] * self.b[self.n - 1]
        for i in range(self.n):
            self.Beta[i] = self.a[self.n - 1] * self.b[self.n - i - 1]
            for j in range(1, i + 1):
                self.Beta[i] -= self.a[self.n - j - 1] * self.Beta[i - j]
        self.B = self.Beta[1:]
        for i in range(self.n - 1):
            for j in range(self.n - 1):
                if (j == i + 1) and (i < self.n - 2):
                    self.A[i][j] = 1
                elif i == self.n - 2:
                    self.A[i][j] = -self.a[j]
        return self.A, self.B, self.C, self.D

    def computeStateEqautions(self):
        """
        Calculation of the dervivative
        of the states
        """
        
        self.Xs = np.zeros((self.k + 1, self.n - 1))
        self.Xs[0] = np.zeros(self.n - 1)
        self.X_temp =  np.zeros(self.n - 1)

        self.Block = np.zeros((6, self.n - 1))
        self.i = 1
        self.j = 0 
        
        ## Calculationg Derivatives
        
        while (True):
            self.t = self.time[self.j]
            self.Block[0] = self.A @ self.X_temp + self.B
            self.X_mod = self.X_temp + self.h * self.Block[0]
            self.Block[1] = self.A @ self.X_mod + self.B 
            self.X_mod = self.X_temp + (self.h / 2) * (self.Block[0] + self.Block[1])
            self.Block[2] = self.A @ self.X_mod + self.B 
            self.X_mod = self.X_temp + (2 * self.h) * (self.Block[2])
            self.Block[3] = self.A @ self.X_mod + self.B 
            self.X_mod = self.X_temp + (self.h / 12) * (5 * self.Block[0] + 8 * self.Block[2] - self.Block[3])
            self.Block[4] = self.A @ self.X_mod + self.B 
            self.X_mod = self.X_temp + (self.h / 3) * (self.Block[0] + self.Block[3] + 4 * self.Block[4])
            self.Block[5] = self.A @ self.X_mod + self.B
            if self.i == self.k + 1:
                break            
            self.Xs[self.i] = self.X_temp + (self.h / 12) * (5 * self.Block[0] + 8 * self.Block[2] - self.Block[3])
            self.i += 1
            self.Xs[self.i] = self.X_temp + (self.h / 3) * (self.Block[0] + 4 * self.Block[4] + self.Block[5])
            self.X_temp = self.Xs[self.i]
            self.i += 1
            self.j+=2
        if (self.inp_type=='impulse'):
            self.u=np.zeros(self.k+1)
            self.u[0]=1
        return self.time, self.Xs, self.u

    def computeOutput(self):
        """
        Compute the ouput 
        using the output equatoin form
        
        """
        self.y = np.zeros((self.k + 1))
        self.y = self.C @ self.Xs.T + self.u*self.D
        if self.inp_type == 'impulse':
            for j in range(self.n-1):
                for i in range(self.k):
                    self.Xs[i,j] = (self.Xs[i+1,j] - self.Xs[i,j])/(self.h)
            self.Xs[-1,:] = self.Xs[-2,:]
        self.y = self.C @ self.Xs.T + self.u*self.D


        self.y[0]=self.y[1]
        return self.y

    def plotOutput(self):
        output_file("ouput.html")
        if self.inp_type == 'impulse':
            self.time = np.linspace(-self.end_t, self.end_t, self.k + 1)
            self.u = (100 * (22/7)) * np.exp(-(self.time*100)**2)
        else:
            self.u1 = self.u.copy()
            self.u1 = np.insert(self.u1, 0, 0.5, axis=0)
            self.u1 = np.insert(self.u1, 0, 0, axis=0)
            self.u1 = np.insert(self.u1, 0, 0, axis=0)
            self.time1 = self.time.copy()
            self.time1 = np.insert(self.time1,0,0,axis=0)
            self.time1 = np.insert(self.time1,0,0,axis=0)
            self.time1 = np.insert(self.time1,0,-100000,axis=0)
        
        ## Plot Input
        
        fig1 = figure(title='Input',
                     plot_height=400, plot_width=600,
                     x_axis_label='Time', y_axis_label='Input',
                     x_minor_ticks=2, x_range=(-5, 5), y_range=(-1,2),
                     toolbar_location=None)
        if self.inp_type == 'impulse':
            fig1.line(x=self.time.tolist(), y=self.u.tolist(),
                     color='red', line_width=1,
                     legend='Input')
            
        else:
            tau = (1-1/np.exp(1)) * self.u[-1]
            fig1.line(x=self.time1.tolist(), y=self.u1.tolist(),
                     color='red', line_width=1,
                     legend='Input')
        tab1 = Panel(child=fig1, title="Input")
        
        self.time = np.linspace(0, self.end_t, self.k + 1)    
        ## Plot Output
        
        fig2 = figure(title='System Response',
                     plot_height=400, plot_width=600,
                     x_axis_label='Time', y_axis_label='Output',
                     x_minor_ticks=2, x_range=(0, self.t), y_range=(np.min(self.y)-np.mean(self.y), np.max(self.y)+np.mean(self.y)),
                     toolbar_location=None)
        fig2.line(x=self.time.tolist(), y=self.y.tolist(),
                 color='blue', line_width=1,
                 legend='Output')
        tab2 = Panel(child=fig2, title="Output")
        tabs = Tabs(tabs=[tab1,tab2])
        show(tabs)
        
    def plotStates(self):
        output_file("States.html")
        tabs = []
        for i in range(1,self.n):
            fig = figure(title='States',
                         plot_height=400, plot_width=600,
                         x_axis_label='Time', y_axis_label='State'+str(i),
                         x_minor_ticks=2, x_range=(0, self.t), y_range=(np.min(self.Xs[:,i-1])-np.abs(np.mean(self.Xs[:,i-1])), np.max(self.Xs[:,i-1])+np.abs(np.mean(self.Xs[:,i-1]))),
                         toolbar_location=None)
            fig.line(x=self.time.tolist(), y=self.Xs[:,i-1].tolist(),
                     color='blue', line_width=1,
                     legend='State'+str(i))
            
            tabs.append(Panel(child=fig, title="State"+str(i)))
        tabs = Tabs(tabs=tabs)
        show(tabs)

